[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/map-reduce.ipynb) [![在 LangChain Academy 中打开](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239947-lesson-3-map-reduce)


# Map-Reduce

## 回顾

我们正在逐步构建一个多智能体研究助手，它会把本课程中的所有模块串联起来。

为了构建这个多智能体助手，我们已经介绍了几个 LangGraph 的可控性主题。

我们刚刚学习了并行化和子图。

## 目标

接下来，我们将[学习 map reduce](https://langchain-ai.github.io/langgraph/how-tos/map-reduce/)。


In [12]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph

In [13]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("OPENAI_API_KEY")
_set_env("DASHSCOPE_API_KEY")

我们将使用 [LangSmith](https://docs.smith.langchain.com/) 来[追踪](https://docs.smith.langchain.com/concepts/tracing)。


In [14]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## 问题

Map-Reduce 操作对于高效的任务拆解和并行处理至关重要。

它包含两个阶段：

(1) `Map` —— 将任务拆分成更小的子任务，并行处理每个子任务。

(2) `Reduce` —— 聚合所有并行子任务的结果。

我们来设计一个系统，完成两件事：

(1) `Map` —— 围绕某个主题生成一组笑话。

(2) `Reduce` —— 从列表中选出最好笑的一个。

我们会使用 LLM 来生成笑话并进行挑选。


In [15]:
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi

# Prompts we will use
subjects_prompt = """Generate a list of 3 sub-topics that are all related to this overall topic: {topic}."""
joke_prompt = """Generate a joke about {subject}"""
best_joke_prompt = """Below are a bunch of jokes about {topic}. Select the best one! Return the ID of the best one, starting 0 as the ID for the first joke. Jokes: \n\n  {jokes}"""

# LLM
# model = ChatOpenAI(model="gpt-4o", temperature=0)
model = ChatTongyi(model="qwen-plus", temperature=0)

## 状态

### 并行生成笑话

首先定义图的入口节点，它将：

* 接收用户输入的主题
* 基于主题生成一系列笑话题目
* 将每个笑话题目发送到我们上面的笑话生成节点

我们的状态包含一个 `jokes` 键，用来累积并行笑话生成得到的笑话。


In [16]:
import operator
from typing import Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel


class Subjects(BaseModel):
    subjects: list[str]


class BestJoke(BaseModel):
    id: int


class OverallState(TypedDict):
    topic: str
    subjects: list
    jokes: Annotated[list, operator.add]
    best_selected_joke: str

生成笑话的题目。


In [17]:
def generate_topics(state: OverallState):
    prompt = subjects_prompt.format(topic=state["topic"])
    response = model.with_structured_output(Subjects).invoke(prompt)
    return {"subjects": response.subjects}

这里是关键：我们使用 [Send](https://langchain-ai.github.io/langgraph/concepts/low_level/#send) 为每个题目生成笑话。

这非常实用！它可以自动并行地为任意数量的题目生成笑话。

* `generate_joke`：图中节点的名称
* `{"subject": s}`：要发送的状态

`Send` 允许你把任意状态传给 `generate_joke`！它不必与 `OverallState` 完全一致。

在这个例子里，`generate_joke` 使用它自己的内部状态，我们通过 `Send` 来填充它。


In [18]:
from langgraph.types import Send


def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": s}) for s in state["subjects"]]

### 笑话生成（map）

现在定义负责创建笑话的节点 `generate_joke`！

我们会把笑话写回 `OverallState` 中的 `jokes` 键。

这个键设置了 reducer，用来合并列表。


In [19]:
class JokeState(TypedDict):
    subject: str


class Joke(BaseModel):
    joke: str


def generate_joke(state: JokeState):
    prompt = joke_prompt.format(subject=state["subject"])
    response = model.with_structured_output(Joke).invoke(prompt)
    return {"jokes": [response.joke]}

### 最佳笑话选择（reduce）

接下来添加逻辑，选出最好笑的笑话。


In [20]:
def best_joke(state: OverallState):
    jokes = "\n\n".join(state["jokes"])
    prompt = best_joke_prompt.format(topic=state["topic"], jokes=jokes)
    response = model.with_structured_output(BestJoke).invoke(prompt)
    return {"best_selected_joke": state["jokes"][response.id]}

## 编译


In [22]:
from IPython.display import Image
from langgraph.graph import END, StateGraph, START

# Construct the graph: here we put everything together to construct our graph
graph = StateGraph(OverallState)
graph.add_node("generate_topics", generate_topics)
graph.add_node("generate_joke", generate_joke)
graph.add_node("best_joke", best_joke)
graph.add_edge(START, "generate_topics")
graph.add_conditional_edges("generate_topics", continue_to_jokes, ["generate_joke"])
graph.add_edge("generate_joke", "best_joke")
graph.add_edge("best_joke", END)

# Compile the graph
app = graph.compile()
# Image(app.get_graph().draw_mermaid_png())

In [23]:
# Call the graph: here we call it to generate a list of jokes
for s in app.stream({"topic": "animals"}):
    print(s)

{'generate_topics': {'subjects': ['Mammals', 'Reptiles', 'Birds']}}
{'generate_joke': {'jokes': ["Why don't birds use Facebook? Because they already have Twitter!"]}}
{'generate_joke': {'jokes': ["Why don't mammals ever get lost? Because they always follow their 'paws'itive' instincts!"]}}
{'generate_joke': {'jokes': ["Why don't reptiles ever get invited to card games? Because they always bring the scales!"]}}
{'best_joke': {'best_selected_joke': "Why don't birds use Facebook? Because they already have Twitter!"}}


## Studio

**⚠️ 免责声明**

自从录制这些视频以来，我们已经更新了 Studio，使其可以在本地运行并在浏览器中打开。现在推荐的方式是以这种形式运行 Studio（而不是视频中展示的桌面应用）。关于本地开发服务器请查看[这里](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)的文档，关于本地 Studio 的运行方式请查看[这里](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。在本模块的 `/studio` 目录中，在终端运行以下命令即可启动本地开发服务器：

```
langgraph dev
```

你应该会看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中访问 Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。

让我们在 Studio UI 中加载上面的图，它由 `module-4/studio/map_reduce.py` 定义，并在 `module-4/studio/langgraph.json` 中进行了配置。
